In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer,DataCollatorForSeq2Seq,Seq2SeqTrainingArguments,Seq2SeqTrainer
import numpy as np
import evaluate

In [9]:
#load dataset
data_files  = {"test":"code_completion_dataset_3k_deduped.json"}
path_to_dataset = '/workspaces/code-rationales/semeru-datasets/semeru/galeras/galeras_se_tasks_dataset_3k_deduplicated'
dataset = load_dataset(path_to_dataset, data_files = data_files, split='test')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/galeras_se_tasks_dataset_3k_deduplicated-f07ebc227c0463f3/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


In [10]:
dataset.column_names

['commit_message',
 'fun_name',
 'ast_errors',
 'ast_levels',
 'vocab_size',
 'n_ast_errors',
 'file_name',
 'n_whitespaces',
 'n_words',
 'n_identifiers',
 'n_ast_nodes',
 'id',
 'complexity',
 'random_cut',
 'nloc',
 'repo',
 'url',
 'code',
 'commit_id',
 'path',
 'token_counts',
 'language']

In [3]:
#load tokenizer and model
checkpoint = '/workspaces/code-rationales/data/codeparrot-small/checkpoints/checkpoint-29000'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [4]:
def preprocess_function(examples):

    model_inputs = tokenizer(examples["random_split"], max_length=1024, truncation=True,padding=True)

    labels = tokenizer(examples["code"], max_length=1024, truncation=True,padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
tokenizer.pad_token = tokenizer.eos_token
 

In [6]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2787 [00:00<?, ? examples/s]

In [7]:
#select a metric
metric = evaluate.load("bleu")

In [8]:
def compute_metrics(preds,labels):
    preds = preds
    labels = labels

    result = metric.compute(predictions=preds, references=labels)
    return result

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model,padding=True)

In [9]:

#Run each examples on model and add it to list

predictions = []
groundTruth  = []



for item in tokenized_dataset:
    input = item['random_split']
    #print(input)
    
    prediction = model.generate(tokenizer.encode(input, max_length=1024, truncation=True, return_tensors="pt"))
    
    predictions.append(tokenizer.decode(prediction[0]))
    groundTruth.append(item['code'])
    

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 176, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 230, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 123, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 200, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Sett

In [10]:
import json
inputF = []
for item in tokenized_dataset:
    inputF.append(item['random_split'])


        
with open('/workspaces/code-rationales/code_rationales/input.json', 'w', encoding='utf-8') as f:
    json.dump(inputF, f, ensure_ascii=False, indent=4)

In [11]:
#compute bleu
result = compute_metrics(predictions,groundTruth)
print(result)

{'bleu': 0.39916739355969627, 'precisions': [0.9944939865063068, 0.9926740468423279, 0.9921300465815356, 0.9919798880732786], 'brevity_penalty': 0.40205455211147045, 'length_ratio': 0.5232403759226544, 'translation_length': 340900, 'reference_length': 651517}
2787


In [15]:
#Sample run on single input

inputS = tokenized_dataset[456]['random_split']

print(inputS)

prompt = "complete the following code snippet " + inputS

pred = model.generate(tokenizer.encode(prompt, max_length=1024, truncation=True, return_tensors="pt"))

print(tokenizer.decode(pred[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 212, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


def execute(self):
        if self._image is None:
            raise BatchJobException(
                "Unable to launch AWS Batch job. No docker image specified."
            )
        if self._iam_role is None:
            raise BatchJobException(
                "Unable to launch AWS Batch job. No IAM role specified."
            )

        # Multinode
        if getattr(self, "num_parallel", 0) > 1:
            num_nodes = self.num_parallel
            main_task_override = copy.deepcopy(self.payload["containerOverrides"])

            # main
            commands = self.payload["containerOverrides"]["command"][-1]
            # add split-index as this worker is also an ubf_task
            commands = commands.replace("[multinode-args]", "--split-index 0")
            main_task_override["command"][-1] = commands

            # secondary tasks
            secondary_task_container_override = copy.deepcopy(
                self.payload["containerOverrides"]
complete the following code 

In [12]:
#save prediction and groundTruth to json
with open('/workspaces/code-rationales/code_rationales/prediction.json', 'w', encoding='utf-8') as f:
    json.dump(predictions, f, ensure_ascii=False, indent=4)

with open('/workspaces/code-rationales/code_rationales/groundTruth.json', 'w', encoding='utf-8') as f:
    json.dump(groundTruth, f, ensure_ascii=False, indent=4)

In [47]:
#args for alternate way to run score

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True
)

In [48]:
#alternate way to run bleu score
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.evaluate()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 230, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer_seq2seq.py:159 in evaluate           │
│                                                                                                  │
│   156 │   │   )                                                                                  │
│   157 │   │   self._gen_kwargs = gen_kwargs                                                      │
│   158 │   │                                                                                      │
│ ❱ 159 │   │   return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix   │
│   160 │                                                                                          │
│   161 │   def predict(                                                                           │
│   162 │   │   self,                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:3053 in evaluate                  │
│                                                                                                  │
│   3050 │   │   start_time = time.time()                                                          │
│   3051 │   │                                                                                     │
│   3052 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3053 │   │   output = eval_loop(                                                               │
│   3054 │   │   │   eval_dataloader,                                                              │
│   3055 │   │   │   description="Evaluation",                                                     │
│   3056 │   │   │   # No point gathering the predictions if there are no metrics, otherwise we d  │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:3245 in evaluation_loop           │
│                                                                                                  │
│   3242 │   │   │   │   │   batch_size = observed_batch_size                                      │
│   3243 │   │   │                                                                                 │
│   3244 │   │   │   # Prediction step                                                             │
│ ❱ 3245 │   │   │   loss, logits, labels = self.prediction_step(model, inputs, prediction_loss_o  │
│   3246 │   │   │   inputs_decode = self._prepare_input(inputs["input_ids"]) if args.include_inp  │
│   3247 │   │   │                                                                                 │
│   3248 │   │   │   if is_torch_tpu_available():                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer_seq2seq.py:295 in prediction_step    │
│                                                                                                  │
│   292 │   │   with torch.no_grad():                                                              │
│   293 │   │   │   if has_labels:                                                                 │
│   294 │   │   │   │   with self.compute_loss_context_manager():                                  │
│ ❱ 295 │   │   │   │   │   outputs = model(**inputs)                                              │
│   296 │   │   │   │   if self.label_smoother is not None:  